# 3D Vision Homework 1
>王子轩 2023011307
`wang-zx23@mails.tsinghua.edu.cn`
## Probelem 1
### Subproblem 1.1
The map $\textit{f}: \mathbb{R}^n \rightarrow \mathbb{R}^3$ is defined by 
$$
f(u, v) = \begin{bmatrix}
a \cos u \sin v  & \\
b \sin u  \sin v  & \\
c \cos v &  
\end{bmatrix}
\text{where} - \pi \leq u \leq  \pi, \quad 0 \leq v \leq \pi
$$
The function $f$ maps the 2D domain to an ellipsoid.
Let $a = 1$, $b = 1$, $c = \frac{1}{2}$. Let $\mathbf{p} = (u, v)$ be a point in the domain of $f$, and let
$\gamma: (-1, 1) \rightarrow \mathbb{R}^2$ be a curve with $\gamma(0) = \mathbf{p}$ and $\gamma'(t) = \mathbf{v}$.
Now we let $\mathbf{p} = (\frac{\pi}{4}, \frac{\pi}{6})$ and $\mathbf{v} = (1, 0)$, we draw the curve $f(\gamma(t))$.

### Solution:
From $\gamma'(t) = (1, 0)\text{ and } \gamma(0) = (\frac{\pi}{4}, \frac{\pi}{6}) $ we get $\gamma(t) = (\frac{\pi}{4} + t, \frac{\pi}{6}) $, thus $$f(\gamma(t)) = \begin{bmatrix}
\cos(\frac{\pi}{4} + t) \sin(\frac{\pi}{6}) \\
\sin(\frac{\pi}{4} + t) \sin(\frac{\pi}{6}) \\
\frac{1}{2} \cos(\frac{\pi}{6})
\end{bmatrix} $$
>Full code using object oriented programming in Python is in src/draw.py 

In [10]:

! python src/draw.py

>Simpler view of the code is list as follows, and the result can be seen in the image.

![draw ellipsoid and curve](assets/image-20250317220956088.png)

In [19]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
a, b, c = 1, 1, 0.5
p = (np.pi/4, np.pi/6)
v = (1, 0)
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Ellipsoid with Curve")
u = np.linspace(-np.pi, np.pi, 50)
v = np.linspace(0, np.pi, 25)
u_grid, v_grid = np.meshgrid(u, v)
x = a * np.cos(u_grid) * np.sin(v_grid)
y = b * np.sin(u_grid) * np.sin(v_grid)
z = c * np.cos(v_grid)
vertices = []
for i in range(len(v)):
    for j in range(len(u)):
        vertices.append([x[i, j], y[i, j], z[i, j]])
vertices = np.array(vertices)

triangles = []
for i in range(len(v)-1):
    for j in range(len(u)-1):
        idx1 = i * len(u) + j
        idx2 = i * len(u) + (j + 1)
        idx3 = (i + 1) * len(u) + j
        idx4 = (i + 1) * len(u) + (j + 1)
        triangles.append([idx1, idx2, idx3])
        triangles.append([idx2, idx4, idx3])
triangles = np.array(triangles)

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.8, 0.8, 0.8]) 

mesh_wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
mesh_wireframe.paint_uniform_color([0.5, 0.5, 0.5])  

t_values = np.linspace(-1, 1, 100)
curve_points = []

for t in t_values:
    u_t = np.pi/4 + t
    v_t = np.pi/6
    x_t = a * np.cos(u_t) * np.sin(v_t)
    y_t = b * np.sin(u_t) * np.sin(v_t)
    z_t = c * np.cos(v_t)
    
    curve_points.append([x_t, y_t, z_t])

curve_points = np.array(curve_points)

line_indices = [[i, i+1] for i in range(len(curve_points)-1)]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(curve_points)
line_set.lines = o3d.utility.Vector2iVector(line_indices)
line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in range(len(line_indices))])  # Red color

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

vis.add_geometry(mesh)
vis.add_geometry(mesh_wireframe)  
vis.add_geometry(line_set)
vis.add_geometry(coordinate_frame)  

opt = vis.get_render_option()
opt.background_color = np.array([1, 1, 1])  
opt.point_size = 5.0
opt.mesh_show_wireframe = True  

vis.run()
vis.destroy_window()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(curve_points)
pcd.paint_uniform_color([0, 1, 0]) 

sample_points = mesh.sample_points_uniformly(number_of_points=3000)
sample_points.paint_uniform_color([0, 0, 0.8])  

o3d.visualization.draw_geometries([mesh, mesh_wireframe, pcd, coordinate_frame, sample_points], 
                                 window_name="Ellipsoid with Curve Points",
                                 mesh_show_wireframe=True)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


### Subpreblem 1.2
Differential map: The differential of a function f at a point p is a linear
map. Equivalent to how we define the differential in class, we can also
define by the gradient of the curve w.r.t. the curve parameter: $Df_{\mathbf{p}(v)} =
f(γ(t))′|t=0.$
### Solution:
#### (a) $Df_{\mathbf{p}(v)}$'s expression:
$Df_{\mathbf{p}(v)} = f(γ(t))′|t=0 = ( f \circ \gamma)'(t)  = f'(\gamma(t)) \gamma'(t) |t=0 = (\frac{\partial f}{\partial \mathbf{p}}) \mathbf{v} = \nabla f \mathbf{v}  
=  (\frac{\partial f}{\partial u}, \frac{\partial f}{\partial v}) \mathbf{v}$ 
thus
$$ 
Df_{\mathbf{p}} 
= 
\begin{bmatrix}
\frac{\partial f}{\partial u} & \frac{\partial f}{\partial v}
\end{bmatrix}
= \begin{bmatrix}
-a\sin u \sin v & a\cos u \cos v \\
b\cos u \sin v & b\sin u \cos v \\
0 & -c\sin v
\end{bmatrix}
= 
\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}
$$
#### (b) Describe the geometry meaning of the differential map
First, $Df_{\mathbf{p}}$ maps vectors in the tangent space of the domain (2D plane) to vectors in the tangent space of the ellipsoid (3D space), and the columns of $Df_{\mathbf{p}}$ form a basis for the tangent plane to the ellipsoid at the point $f(\mathbf{p})$. The matrix $\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}$ describes how infinitesimal vectors near $\mathbf{p}$ are transformed when mapped to the ellipsoid. Second, the differential map $Df_{\mathbf{p}}$ represents the best linear approximation of the function $f$ at point $\mathbf{p}$.
#### (c) Draw Dfp(v) on the ellipsoid
$\mathbf{p}=( \frac{π}{4}, \frac{π}{6}) and \mathbf{v} = (1, 0)$ and we konw that 
$Df_{\mathbf{p}} =
\begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}$ Thus 
$$
Df_{\mathbf{p}} \mathbf{v} 
= \begin{bmatrix}
-\sin u \sin v & \cos u \cos v \\
\cos u \sin v & \sin u \cos v \\
0 & -\frac{1}{2}\sin v
\end{bmatrix}
\begin{bmatrix}
1 \\
0
\end{bmatrix}
= 
\begin{bmatrix}
-\sin u \\
\cos u \\
0
\end{bmatrix}
$$
The implemention of full code is in `src/draw.py`, and the class `DifferentialMap` is used to draw the $D_f(v)$ in both directions of the tangent plane.
Run `! python draw.py` you can see the result that the $D_f(v)$ is drawn in both directions of the tangent plane. The Red arrow is the direction of $\mathbf{v} = (1,0)$ and the Blue arrow is the direction of $\mathbf{v} = (0, 1)$.  In the following image, the Red arrow is the direction of $\mathbf{v} = (1,0)$ and the Blue arrow is the direction of $\mathbf{v} = (0, 1)$. See the second image:
![](\assets\image-20250318000822800.png)
![](assets\image-20250318001837911.png)
Furthermore, we can draw the $D_f(v)$ in any direction of the tangent plane, given the direction of $\mathbf{v}$.
For example, we draw the $D_f(v)$ in the direction of $\mathbf{v} = (1,1)$ and $(2, -1)$, we can see that the $D_f(v)$ is perpendicular to the tangent plane as expected. See the third image:
![](assets\image-20250318002530291.png)

Implementation code for the Df(p)

```python
class DifferentialMap:
    def __init__(self, ellipsoid, u, v):
        self.ellipsoid = ellipsoid
        self.u = u
        self.v = v
        
    def compute_dfp(self):
        """Calculate Df_p @ (u,v)"""
        a, b, c = self.ellipsoid.a, self.ellipsoid.b, self.ellipsoid.c
        u, v = self.u, self.v
        dfp = np.array([
            [-a * np.sin(u) * np.sin(v), a * np.cos(u) * np.cos(v)],
            [b * np.cos(u) * np.sin(v), b * np.sin(u) * np.cos(v)],
            [0, -c * np.sin(v)]
        ])
        return dfp
    
    def apply_dfp_to_vector(self, v_direction):
        """Apply Df_p to a specific direction vector v"""
        dfp = self.compute_dfp()
        v_direction = np.array(v_direction)
        v_direction = v_direction / np.linalg.norm(v_direction)
        result = dfp @ v_direction
        return result
    
    def create_differential_vector(self, v_direction, scale=0.2, color=[1, 0, 0]):
        """Create a visualization of Df_p(v) for a specific direction v"""
        x = self.ellipsoid.a * np.cos(self.u) * np.sin(self.v)
        y = self.ellipsoid.b * np.sin(self.u) * np.sin(self.v)
        z = self.ellipsoid.c * np.cos(self.v)
        base_point = np.array([x, y, z])
        direction = self.apply_dfp_to_vector(v_direction)
        direction = direction / np.linalg.norm(direction) * scale
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(
            radius=0.01,
            height=np.linalg.norm(direction)
        )
        cone = o3d.geometry.TriangleMesh.create_cone(
            radius=0.02,
            height=0.05
        )
        z_axis = np.array([0, 0, 1])
        if not np.allclose(direction, z_axis) and not np.allclose(direction, -z_axis):
            rotation_axis = np.cross(z_axis, direction)
            rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
            angle = np.arccos(np.dot(z_axis, direction/np.linalg.norm(direction)))
            R = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * angle)
            cylinder.rotate(R, center=[0, 0, 0])
            cone.rotate(R, center=[0, 0, 0])
        cylinder.translate(base_point + direction/2)
        cone.translate(base_point + direction)
        cylinder.paint_uniform_color(color)
        cone.paint_uniform_color(color)
        return [cylinder, cone]
    
    def create_differential_vectors(self, scale=0.2, directions=None):
        """Create visualizations for multiple direction vectors"""
        if directions is None:
            # Default: standard basis vectors
            directions = [
                ([1, 0], [1, 0, 0]),  # u direction, red color
                ([0, 1], [0, 0, 1])   # v direction, blue color
            ]
        arrows = []
        for v_dir, color in directions:
            arrow_parts = self.create_differential_vector(v_dir, scale, color)
            arrows.extend(arrow_parts)
        
        return arrows
```

In [23]:
! python src/draw.py